In [1]:
from datasets import load_from_disk

In [2]:
dataset = load_from_disk('./wikipedia_ja_20240801')

Loading dataset from disk:   0%|          | 0/80 [00:00<?, ?it/s]

In [3]:

# dataset のうち，ランダムにな1000件をvalidation，別のランダムな1000件をtest，その他のデータをtrainに分割
dataset_dict = dataset.shuffle(seed=42).train_test_split(test_size=20, shuffle=True)
train_dataset = dataset_dict['train']
valid_and_test_dataset_dict = dataset_dict['test'].train_test_split(test_size=10, shuffle=True)
valid_dataset = valid_and_test_dataset_dict['train']
test_dataset = valid_and_test_dataset_dict['test']

# 改めてDatasetDictを作成
from datasets import DatasetDict
dataset = DatasetDict({'train': train_dataset, 'validation': valid_dataset, 'test': test_dataset})

In [4]:
from transformers import MBartForConditionalGeneration, AutoTokenizer

model_name = "ku-nlp/bart-base-japanese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [5]:
from transformers import pipeline

generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer, max_new_tokens=60)
generated = generator(dataset['train'][0]['text'])
print(generated)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[{'generated_text': '宗資にはその後もの加増が繰り返され 月の日の日に加増されて 月 の 日 の 日 に加増 されて 月の月の日 に 。 宗資 （ じ ） にはそ の こと に は 。'}]


In [6]:
dataset["train"][0]["text"]

'宗資にはその後も贔屓の加増が繰り返され、元禄５年（１６９２年）、本庄邸への２度目の“将軍お成り”の際に４万石に加増されて常陸笠間藩へ移封された。'

In [7]:
tokenizer.decode(dataset["train"][0]['input_ids'])

'<s> 宗資にはその後も<unk>の加増が繰り返され、元禄５年（１６９２年）、本庄邸への２度目の“将軍お成り”の際に４万石に加増されて常陸笠間藩へ移封された。</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [8]:
tokenizer.decode(dataset["train"][0]["labels"])

'<s> ＾ ｓ ｏ ］ ｏ ＃ ｓｈ ｉ ］ ｎ ｉ ｗ ａ ＃ ｓ ｏ ［ ｎ ｏ ｇ ｏ ｍ ｏ ＃ ｈ ｉ ］ ｉ ｋ ｉ ｎ ｏ ＃ ｋ ａ ［ ｚ ｏ ｏ ｇ ａ ＃ ｋ ｕ ［ ｒ ｉ ｋ ａ ］ ｅ ｓ ａ ｒ ｅ ＿ ｇ ｅ ］ Ｎ ｒ ｏ ｋ ｕ ＃ ｇ ｏ ］ ｎ ｅ Ｎ ＿ ｓ ｅ ］ Ｎ ＃ ｒ ｏ ［ ｃｌ ｐｙ ａ ｋ ｕ ＃ ｋｙ ｕ ］ ｕ ｊ ｕ ｕ ＃ ｎ ｉ ］ ｎ ｅ Ｎ ＿ ｈ ｏ ［ Ｎ ｊ ｏ ｏ ｔ ｅ ｅ ｅ ｎ ｏ ＃ ｎ ｉ ［ ｄ ｏ ｍ ｅ ｎ ｏ ＿ ｓｈ ｏ ［ ｏ ｇ ｕ Ｎ ｏ ］ ｎ ａ ｒ ｉ ＿ ｎ ｏ ＃ ｓ ａ ］ ｉ ｎ ｉ ＃ ｙ ｏ ［ Ｎ ｍ ａ Ｎ ］ ｚ ｅ ｋ ｉ ｎ ｉ ＃ ｋ ａ ［ ｚ ｏ ｏ ＃ ｓ ａ ［ ｒ ｅ ｔ ｅ ＃ ｈ ｉ ］ ｔ ａ ｃｈ ｉ ＃ ｋ ａ ［ ｓ ａ ｍ ａ ］ ｈ ａ Ｎ ｅ ＃ ｉ ［ ｈ ｏ ｏ ＃ ｓ ａ ［ ｒ ｅ ｔ ａ ＄</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [10]:
exp_name = "mbart_japanese_wikipedia"
output_dir = f"exp/{exp_name}/results"
logging_dir = f"exp/{exp_name}/logs"

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=3,             # 最大3エポックとする
    per_device_train_batch_size=8,  # バッチサイズ
    auto_find_batch_size=True,      # バッチサイズを自動で見つける

    weight_decay=0.01,              # 重み減衰
    learning_rate=2e-5,             # 学習率
    warmup_steps=1000,              # ウォームアップステップ数

    evaluation_strategy="steps",    # 評価はステップごとに行う
    eval_steps=500,                 # 500ステップごとに評価を行う
    # metric_for_best_model="accuracy", # 最良のモデルの評価指標
    # greater_is_better=True,           # 評価指標が大きいほど良い場合はTrue

    output_dir=output_dir,          # モデルの保存先
    save_strategy="steps",          # モデルの保存はエポックごとに行う
    save_total_limit=3,             # 保存するモデルの数)

    logging_dir=logging_dir,        # ログの保存先
    logging_strategy="steps",       # ログの保存はエポックごとに行う
    logging_steps=500,              # 100ステップごとにログを出力する

    load_best_model_at_end=True,    # 最良のモデルを最後にロードする
)

/home/fujie/.conda/envs/py311/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3, 
    early_stopping_threshold=0.001)

In [17]:
from transformers import Trainer #, DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    # data_collator=data_collator,
    # compute_metrics=compute_metrics,
    # callbacks=[early_stopping],
)

In [18]:
enable_training = True
if enable_training:
    trainer.train()

Step,Training Loss,Validation Loss
500,5.930500,2.731376


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


KeyboardInterrupt: 

In [1]:
enable_model_loading = False
output_dir = f"exp/{exp_name}/results"
model_path = f"{output_dir}/checkpoint-58940"
if enable_model_loading:
    model = MBartForConditionalGeneration.from_pretrained(model_path)

NameError: name 'exp_name' is not defined

In [123]:
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [126]:
split = 'validation'
i = 1
predicted = generator(dataset[split][i]["text"])
print(f"input: {dataset[split][i]['text']}")
print(f"target: {dataset[split][i]['phonemes']}")
print(f"predicted: {predicted[0]['generated_text']}")

input: 「履歴書ってパソコンで作ったらダメなの？」パソコンＶＳ手書きについて考える【話題】
target: ＾ ｒ ｉ ［ ｒ ｅ ｋ ｉ ｓｈ ｏ ｃｌ ｔ ｅ ＃ ｐ ａ ［ ｓ ｏ ｋ ｏ Ｎ ｄ ｅ ＃ ｔｓ ｕ ［ ｋ ｕ ］ ｃｌ ｔ ａ ｒ ａ ＃ ｄ ａ ［ ｍ ｅ ］ ｎ ａ ｎ ｏ ＿ ｐ ａ ［ ｓ ｏ ｋ ｏ Ｎ ｂ ａ ａ ｓ ａ ｓ ｕ ｔ ｅ ］ ｇ ａ ｋ ｉ ｎ ｉ ＃ ｔｓ ｕ ］ ｉ ｔ ｅ ＃ ｋ ａ ［ Ｎ ｇ ａ ］ ｅ ｒ ｕ ＿ ｗ ａ ［ ｄ ａ ｉ ＄
predicted: ＾ ｒ ｅ ［ ｋ ｉ ｒ ｅ ］ ｓｈ ｉ ｔ ｅ ＃ ｐ ａ ［ ｓ ｏ ｋ ｏ Ｎ ｄ ｅ ＃ ｔｓ ｕ ［ ｋ ｕ ｃｌ ｔ ａ ］ ｒ ａ ＃ ｄ ａ ］ ｍ ｅ ｎ ａ ｎ ｏ ＿ ｐ ａ ］ ｓ ｏ Ｎ ｂ ｕ ｉ ＃ ｅ ］ ｓ ｕ ＃ ｓｈ ｕ ［ ｇ ａ ｋ ｉ ｎ ｉ ＃ ｔ ｅ ［ ｃｌ ｃｈ ｉ ＃ ｋ ａ ［ ｎ ａ ｅ ］ ｒ ｕ ＿ ｗ ａ ［ ｄ ａ ｉ ＄


In [132]:
generator("知らない人に道を聞いてみました")

[{'generated_text': '＾ ｓｈ ｉ ［ ｒ ａ ｎ ａ ｉ ＃ ｈ ｉ ［ ｔ ｏ ｎ ｉ ＃ ｄ ｏ ［ ｏ ｏ ＃ ｋ ｉ ［ ｉ ｔ ｅ ＃ ｍ ｉ ［ ｍ ａ ］ ｓｈ ｉ ｔ ａ ＄'}]